# Постановка задачи

На сегодняшний день машинное обучение решает абсолютно разные задачи практически во всех областях нашей жизни. Платформа DataDriven организовывает контесты по машинному обучению для решения реальных задач общественных и некоммерческих организаций по всему миру, применяя результаты победителей. Данный проект призван решить проблемы водоснабжения в стране Танзания в Восточной Африке. Вода в Танзании добывается с помощью водокачек, и они часто выходят из строя, оставляя поселения без воды. Задача участников - по параметрам и статусу некоторого множества водокачек определить возможность поломки и необходимость обслуживания других водокачек. 

В данной задаче мы не только сделаем предсказания о состоянии водокачек, но и оценим, как качество данных влияет на предсказание.

Ссылка на контест: https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/



# Описание метода

Задача будет решаться с помощью градиентного бустинга - будем использовать одноимённый метод GradientBoosting из пакета sklearn.

## Практическая суть задачи

Пусть у нас существует некоторый набор пар {(𝑥𝑖,𝑦𝑖)}, i=1..n, где x - это признак, y - целевая переменная. Задача сводится к тому, чтобы восстановить функцию y=F(x) - функцию, минимизирующую функцию потерь L(x, F(x))

## Градиентный бустинг

Метод градиентного бустинга позволяет построить итоговую модель F(x) как взвешенную функцию слабых моделей Fm(x), m = 1..M. К каждой предыдущей сумме Fm-1(x) добавляется частная производная от функции L(x, Fm(x)) по Fm(x) со знаком минус - для минимизации потерь. Назовём эту производную hm(x). Необходимо также, чтобы функция L(x, Fm(x)) была дифференцируема в достаточно большой области. 

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import GradientBoostingClassifier

Для работы будем использовать библиотеку Pandas. Выведем следующие данные:
TrainingX - множество признаков - набор данных о характеристиках водокачек
TrainingY - множество целевых переменных - набор данных о состоянии водокачек
TestX - множество признаков - набор данных о характеристиках водокачек, по которым требуется совершить предсказание.

In [2]:
TrainingX = pd.read_csv('./contest/trainingsetvalues.csv', sep=',')
TrainingY = pd.read_csv('./contest/trainingsetlabels.csv', sep=',')
TestX = pd.read_csv('./contest/testsetvalues.csv', sep=',')

Посмотрим, что представляют из себя данные о характеристиках водокачек:

In [3]:
print(type(TrainingX))

<class 'pandas.core.frame.DataFrame'>


In [4]:
TrainingX.head()

id  amount_tsh date_recorded        funder  gps_height     installer  \
0  69572      6000.0    2011-03-14         Roman        1390         Roman   
1   8776         0.0    2013-03-06       Grumeti        1399       GRUMETI   
2  34310        25.0    2013-02-25  Lottery Club         686  World vision   
3  67743         0.0    2013-01-28        Unicef         263        UNICEF   
4  19728         0.0    2011-07-13   Action In A           0       Artisan   

   longitude   latitude              wpt_name  num_private  ... payment_type  \
0  34.938093  -9.856322                  none            0  ...     annually   
1  34.698766  -2.147466              Zahanati            0  ...    never pay   
2  37.460664  -3.821329           Kwa Mahundi            0  ...   per bucket   
3  38.486161 -11.155298  Zahanati Ya Nanyumbu            0  ...    never pay   
4  31.130847  -1.825359               Shuleni            0  ...    never pay   

  water_quality quality_group      quantity  quantity_group  \
0          soft          good        enough          enough   
1          soft          good  insufficient    insufficient   
2          soft          good        enough          enough   
3          soft          good           dry             dry   
4          soft          good      seasonal        seasonal   

                 source           source_type  source_class  \
0                spring                spring   groundwater   
1  rainwater harvesting  rainwater harvesting       surface   
2                   dam                   dam       surface   
3           machine dbh              borehole   groundwater   
4  rainwater harvesting  rainwater harvesting       surface   

               waterpoint_type waterpoint_type_group  
0           communal standpipe    communal standpipe  
1           communal standpipe    communal standpipe  
2  communal standpipe multiple    communal standpipe  
3  communal standpipe multiple    communal standpipe  
4           communal standpipe    communal standpipe  

[5 rows x 40 columns]

# Пояснения к данным

Перечень столбцов с расшифровкой:
amount_tsh - Общий статический напор (количество воды, доступное для водокачки)
date_recorded - Дата ввода данных
funder - Кто финансировал колодец
gps_height - Глубина скважины
installer - Организация, вырывшая скважину
longitude - GPS-координата
latitude - GPS-координата
wpt_name - Название водоема, если он есть
num_private -
basin - Географическое расположение водоёма

subvillage - Географическое положение населённого пункта
region - Географическое положение региона
region_code - Код региона
district_code - Код района
lga - Географическое положение
ward - Географическое положение
population - Население вокруг скважины
public_meeting - Верно/Ложно
recorded_by - Групповой ввод этой строки данных
scheme_management - Кто управляет водокачкой
scheme_name - Кто управляет водокачкой
permit - Разрешен ли водозабор
construction_year - Год постройки водозабора
extraction_type - Тип добычи воды, которую использует водонапорная точка
extraction_type_group - Подвид добычи воды, которую использует водонапорная точка
extraction_type_class - Класс добычи воды, которую использует водонапорная точка
management - Как управляется водокачка
management_group - Как управляется водокачка
payment - Сколько стоит вода
payment_type - Сколько стоит вода
water_quality - Качество воды
quality_group - Качество воды
quantity - Количество воды
quantity_group - Количество воды
source - Источник воды
source_type - Источник воды
source_class - Источник воды
waterpoint_type - Вид водоема
waterpoint_type_group - Вид водоема


# Распределение данных по параметрам строковых типов
Чтобы включить в модель такие признаки, необходимо, чтобы было достаточно данных по каждому из них, и чтобы данные были достаточно равномерно распределены по ним. Для начала посмотрим, какие типы столбцов и в каком количестве у нас имеются:

In [5]:
TrainingX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

Как мы видим,  строковых значений в таблице 30, целочисленных - 7, вещественных - 3. Посмотрим на таблицу, содержащую только строковые типы данных:

In [6]:
TrainingX.select_dtypes(include=['object'])

date_recorded           funder     installer              wpt_name  \
0        2011-03-14            Roman         Roman                  none   
1        2013-03-06          Grumeti       GRUMETI              Zahanati   
2        2013-02-25     Lottery Club  World vision           Kwa Mahundi   
3        2013-01-28           Unicef        UNICEF  Zahanati Ya Nanyumbu   
4        2011-07-13      Action In A       Artisan               Shuleni   
...             ...              ...           ...                   ...   
59395    2013-05-03  Germany Republi           CES   Area Three Namba 27   
59396    2011-05-07      Cefa-njombe          Cefa     Kwa Yahona Kuvala   
59397    2011-04-11              NaN           NaN               Mashine   
59398    2011-03-08            Malec          Musa                Mshoro   
59399    2011-03-23       World Bank         World       Kwa Mzee Lugawa   

                         basin    subvillage       region             lga  \
0                   Lake Nyasa      Mnyusi B       Iringa          Ludewa   
1                Lake Victoria       Nyamara         Mara       Serengeti   
2                      Pangani       Majengo      Manyara       Simanjiro   
3      Ruvuma / Southern Coast    Mahakamani       Mtwara        Nanyumbu   
4                Lake Victoria    Kyanyamisa       Kagera         Karagwe   
...                        ...           ...          ...             ...   
59395                  Pangani      Kiduruni  Kilimanjaro             Hai   
59396                   Rufiji      Igumbilo       Iringa          Njombe   
59397                   Rufiji     Madungulu        Mbeya         Mbarali   
59398                   Rufiji        Mwinyi       Dodoma        Chamwino   
59399              Wami / Ruvu  Kikatanyemba     Morogoro  Morogoro Rural   

                   ward public_meeting  ... payment_type water_quality  \
0              Mundindi           True  ...     annually          soft   
1                 Natta            NaN  ...    never pay          soft   
2               Ngorika           True  ...   per bucket          soft   
3              Nanyumbu           True  ...    never pay          soft   
4            Nyakasimbi           True  ...    never pay          soft   
...                 ...            ...  ...          ...           ...   
59395  Masama Magharibi           True  ...   per bucket          soft   
59396            Ikondo           True  ...     annually          soft   
59397           Chimala           True  ...      monthly      fluoride   
59398      Mvumi Makulu           True  ...    never pay          soft   
59399        Ngerengere           True  ...   on failure         salty   

      quality_group      quantity quantity_group                source  \
0              good        enough         enough                spring   
1              good  insufficient   insufficient  rainwater harvesting   
2              good        enough         enough                   dam   
3              good           dry            dry           machine dbh   
4              good      seasonal       seasonal  rainwater harvesting   
...             ...           ...            ...                   ...   
59395          good        enough         enough                spring   
59396          good        enough         enough                 river   
59397      fluoride        enough         enough           machine dbh   
59398          good  insufficient   insufficient          shallow well   
59399         salty        enough         enough          shallow well   

                source_type source_class              waterpoint_type  \
0                    spring  groundwater           communal standpipe   
1      rainwater harvesting      surface           communal standpipe   
2                       dam      surface  communal standpipe multiple   
3                  borehole  groundwater  communal standpipe multiple   
4      rainwater harvesti

## Практическая оценка качества данных
Для начала попробуем оценить качество данных, построив модель напрямую. Оценим точность предсказания, разделив данные на две части: на одной обучим модель, по второй оценим точность.

Делим данные на два датафрейма:

In [113]:
TrainX1 = TrainingX[:29700]
TrainY1 = TrainingY[:29700]
TestX2 = TrainingX[29700:]
TestY2 = TrainingY[29700:]
TestY = TrainingY[29700:]

Создаём объект модели:

In [114]:
model = GradientBoostingClassifier(
        n_estimators=500,
        max_depth=5,
        #subsample=0.5,
        max_features="auto"
        # random_state=seed
    )

In [115]:
TestY2

id status_group
29700  62962            1
29701  35938            1
29702   5107            0
29703  64811            1
29704   8958            1
...      ...          ...
59395  60739            0
59396  27263            0
59397  37057            0
59398  31282            0
59399  26348            0

[29700 rows x 2 columns]

Включаем в выборку все столбцы, приводимые к float напрямую, и приводим типы данных:

In [116]:
TrainX1.dtypes == object


id                       False
amount_tsh               False
date_recorded             True
funder                    True
gps_height               False
installer                 True
longitude                False
latitude                 False
wpt_name                  True
num_private              False
basin                     True
subvillage                True
region                    True
region_code              False
district_code            False
lga                       True
ward                      True
population               False
public_meeting            True
recorded_by               True
scheme_management         True
scheme_name               True
permit                    True
construction_year        False
extraction_type           True
extraction_type_group     True
extraction_type_class     True
management                True
management_group          True
payment                   True
payment_type              True
water_quality             True
quality_

Все столбцы, типом которых НЕ является object, мы можем привести к типу float. 

In [117]:
ctrainX = TrainX1[['id',
 'amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'region_code',
 'district_code',
 'population',
 'construction_year'
                  ]]
#ctrainX = ctrainX.values()
ctrainX = ctrainX.to_numpy()
ctrainX = ctrainX.astype(float)
ctrainX

array([[6.9572e+04, 6.0000e+03, 1.3900e+03, ..., 5.0000e+00, 1.0900e+02,
        1.9990e+03],
       [8.7760e+03, 0.0000e+00, 1.3990e+03, ..., 2.0000e+00, 2.8000e+02,
        2.0100e+03],
       [3.4310e+04, 2.5000e+01, 6.8600e+02, ..., 4.0000e+00, 2.5000e+02,
        2.0090e+03],
       ...,
       [6.0922e+04, 0.0000e+00, 0.0000e+00, ..., 4.0000e+00, 0.0000e+00,
        0.0000e+00],
       [5.6395e+04, 0.0000e+00, 0.0000e+00, ..., 1.0000e+00, 0.0000e+00,
        0.0000e+00],
       [7.2779e+04, 0.0000e+00, 1.5060e+03, ..., 7.0000e+00, 1.4000e+02,
        2.0010e+03]])

In [118]:
TrainY1.replace(to_replace="functional", value=0, inplace=True)
TrainY1.replace(to_replace="non functional", value=1, inplace=True)
TrainY1.replace(to_replace="functional needs repair", value=2, inplace=True)

ctrainY = TrainY1
ctrainY = ctrainY.to_numpy()
ctrainY = ctrainY[:, 1]
ctrainY = ctrainY.astype(float)
ctrainY

c:\python3\lib\site-packages\pandas\core\frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


array([0., 0., 0., ..., 1., 1., 0.])

In [119]:
ctestX = TestX2[['id',
 'amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'region_code',
 'district_code',
 'population',
 'construction_year'
                  ]]
ctestX = ctestX.to_numpy()
ctestX = ctestX.astype(float)


Строим модель:

In [120]:
model.fit(ctrainX,ctrainY)

#predictions = model.predict_proba(testX)[:,1]
predictions = model.predict_proba(ctestX)
predictions

array([[0.16324187, 0.79217712, 0.04458101],
       [0.49407471, 0.41001585, 0.09590944],
       [0.93835256, 0.04746945, 0.01417799],
       ...,
       [0.5522829 , 0.19316773, 0.25454937],
       [0.45815469, 0.4832003 , 0.05864501],
       [0.40366304, 0.56320941, 0.03312755]])

Мы получили таблицу вероятностей каждого из трех состояний. Выберем максимально вероятные состояния:

In [121]:
answ = predictions[:, 0].astype(int)
for ii in range (0, len(predictions)):
    answ[ii] = predictions[ii].argmax();
answ

array([1, 0, 0, ..., 0, 1, 1])

Теперь запишем их в датафрейм и сравним с действительными данными:

In [122]:
TestY['status_group'] = answ
TestY

c:\python3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


id  status_group
29700  62962             1
29701  35938             0
29702   5107             0
29703  64811             0
29704   8958             1
...      ...           ...
59395  60739             0
59396  27263             0
59397  37057             0
59398  31282             1
59399  26348             1

[29700 rows x 2 columns]

In [123]:
TestY2

id status_group
29700  62962            1
29701  35938            1
29702   5107            0
29703  64811            1
29704   8958            1
...      ...          ...
59395  60739            0
59396  27263            0
59397  37057            0
59398  31282            0
59399  26348            0

[29700 rows x 2 columns]

In [124]:
TestY2.replace(to_replace="functional", value=0, inplace=True)
TestY2.replace(to_replace="non functional", value=1, inplace=True)
TestY2.replace(to_replace="functional needs repair", value=2, inplace=True)
TestY2

c:\python3\lib\site-packages\pandas\core\frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


id status_group
29700  62962            1
29701  35938            1
29702   5107            0
29703  64811            1
29704   8958            1
...      ...          ...
59395  60739            0
59396  27263            0
59397  37057            0
59398  31282            0
59399  26348            0

[29700 rows x 2 columns]

In [125]:
(TestY2['status_group'] == TestY['status_group']).sum()

20287

In [131]:
acc = 100 * (TestY2['status_group'] == TestY['status_group']).sum() / len(TestY2)
print(acc,'%')

68.3063973063973 %


Точность предсказания составила 68.3%! Это неплохо, учитывая, что данные ещё никак не оптимизировались. Займёмся их оптимизацией.

## Оптимизация данных
Для начала избавимся от дубликатов в фрейме:

In [167]:
DropTrainX = TrainingX.copy()
DropTrainX = DropTrainX.drop_duplicates(subset = ['amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'],keep = 'first')
DropTrainX

id  amount_tsh date_recorded           funder  gps_height  \
0      69572      6000.0    2011-03-14            Roman        1390   
1       8776         0.0    2013-03-06          Grumeti        1399   
2      34310        25.0    2013-02-25     Lottery Club         686   
3      67743         0.0    2013-01-28           Unicef         263   
4      19728         0.0    2011-07-13      Action In A           0   
...      ...         ...           ...              ...         ...   
59395  60739        10.0    2013-05-03  Germany Republi        1210   
59396  27263      4700.0    2011-05-07      Cefa-njombe        1212   
59397  37057         0.0    2011-04-11              NaN           0   
59398  31282         0.0    2011-03-08            Malec           0   
59399  26348         0.0    2011-03-23       World Bank         191   

          installer  longitude   latitude              wpt_name  num_private  \
0             Roman  34.938093  -9.856322                  none            0   
1           GRUMETI  34.698766  -2.147466              Zahanati            0   
2      World vision  37.460664  -3.821329           Kwa Mahundi            0   
3            UNICEF  38.486161 -11.155298  Zahanati Ya Nanyumbu            0   
4           Artisan  31.130847  -1.825359               Shuleni            0   
...             ...        ...        ...                   ...          ...   
59395           CES  37.169807  -3.253847   Area Three Namba 27            0   
59396          Cefa  35.249991  -9.070629     Kwa Yahona Kuvala            0   
59397           NaN  34.017087  -8.750434               Mashine            0   
59398          Musa  35.861315  -6.378573                Mshoro            0   
59399         World  38.104048  -6.747464       Kwa Mzee Lugawa            0   

       ... payment_type water_quality quality_group      quantity  \
0      ...     annually          soft          good        enough   
1      ...    never pay          soft          good  insufficient   
2      ...   per bucket          soft          good        enough   
3      ...    never pay          soft          good           dry   
4      ...    never pay          soft          good      seasonal   
...    ...          ...           ...           ...           ...   
59395  ...   per bucket          soft          good        enough   
59396  ...     annually          soft          good        enough   
59397  ...      monthly      fluoride      fluoride        enough   
59398  ...    never pay          soft          good  insufficient   
59399  ...   on failure         salty         salty        enough   

       quantity_group                source           source_type  \
0              enough                spring                spring   
1        insufficient  rainwater harvesting  rainwater harvesting   
2              enough                   dam                   dam   
3                 dry           machine dbh              borehole   
4            seasonal  rainwater harvesting  rainwater harvesting   
...               ...                   ...                   ...   
59395          enough                spring                spring   
59396          enough                 river            river/lake   
59397          enough           machine dbh              borehole   
59398    insufficient          shallow well          shallow well   
59399          enough          shallow well          shallow well   

       source_class              waterpoint_type waterpoint_type_group  
0       groundwater           communal standpipe    communal standpipe  
1           surface           communal standpipe    communal standpipe  
2           surface  communal standpipe multiple    communal standpipe  
3       groundwater  communal standpipe multiple    communal standpipe  
4           surface           communal standpipe    communal standpipe  
...             ...                          ...                   ...  
59395   groundwater        

## Дата ввода данных
Оценим разброс дат ввода данных. Чем больше этот разброс, тем целесообразнее учитывать возраст водокачек на момент записи, нежели дату постройки.